# Calculation of the cross entropy loss (NLL) for a classification tasks


**Goal:** In this notebook you will use Keras to set up a CNN for classification of MNIST images and calculate the cross entropy before the CNN was trained. You will first calculate the cross entropy loss for a binary classification problem and then for a classification problem with ten classes. You will use basic numpy functions to calculate the loss that is expected from random guessing and see that an untrained CNN is not better than guessing.

**Usage:** Before working through this notebook we recommend to read chapter 4.2. The idea of the notebook is that you try to understand the provided code by running it, checking the output and playing with it by slightly changing the code and rerunning it. 

**Dataset:** You work with the MNIST dataset. You have 60'000 28x28 pixel greyscale images of digits (0-9).

**Content:**
* load the original MNIST data 
* create a subset the of the data to make it binary classification problem
* define a CNN in Keras
* evaluation of the cross entropy loss function of the untrained CNN for only two classes
* evaluation of the cross entropy loss function of the untrained CNN for all classes
* implement the loss function yourself using the predicted probabilities and numpy


| [open in colab](https://colab.research.google.com/github/tensorchiefs/dl_book/blob/master/chapter_04/nb_ch04_02.ipynb)



#### Imports

First you load all the required libraries. 

In [1]:
# load required libraries:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.metrics import confusion_matrix

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras import optimizers






#### Loading and preparing the MNIST data
You download the MNIST data, normalize the pixel-values to be between 0 and 1 and create  a sub-dataset which only contains images with the labels 0 and 1.

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

X_train=x_train / 255 #divide by 255 so that they are in range 0 to 1
X_train=np.reshape(X_train, (X_train.shape[0],28,28,1))
Y_train=tensorflow.keras.utils.to_categorical(y_train,10) # one-hot encoding

# define sub data containing only images with 0 or 1
idx = (y_train==0)|(y_train==1)

X_train_sub = X_train[idx]
Y_train_sub = y_train[idx]
Y_train_sub=tensorflow.keras.utils.to_categorical(Y_train_sub,2) # one-hot encoding

Y_train.shape, X_train.shape, Y_train_sub.shape, X_train_sub.shape

11493376/11490434 [==============================] - 8s 1us/step


((60000, 10), (60000, 28, 28, 1), (12665, 2), (12665, 28, 28, 1))

## CNN model

You use the same CNN model as in chapter 2. First you will use it to evaluate the loss for only two classes (0 and 1)  and then for all ten classes of the MNIST digits.

In [3]:
# here you define hyperparameter of the CNN
batch_size = 128
nb_classes = 2  # for the sub data you only have 2 classes
img_rows, img_cols = 28, 28
kernel_size = (3, 3)
input_shape = (img_rows, img_cols, 1)
pool_size = (2, 2)

In [4]:
# define CNN with 2 convolution blocks and 2 fully connected layers
model = Sequential()

model.add(Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(8, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(16, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(16,kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(40))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# compile model and intitialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [5]:
# summarize model along with number of model weights
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 8)         80        
_________________________________________________________________
activation (Activation)      (None, 28, 28, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 16)        0

### Exercise 1: Evaluation of the untrained model using Keras

<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/paper-pen.png" width="60" align="left" />  


*Exercise 1: Compute the cross entropy loss with Keras and compare the result with the value that you would expect when you have a classification problem with two classes. Remember that the network is untrained and the predictions are basically just random guesses.*  

You best use the function model.evaluate(), to get the cross entropy loss for the untrained network. The input for this function are the images and the corresponding true labels. The function returns the cross entropy loss and the accuracy of the predictions. Note that you use the sub dataset with only two classes (0 and 1). 




In [6]:
# Write your code here

Scroll down to see the solution.

</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>

In [7]:
model.evaluate(X_train_sub, Y_train_sub)

12665/12665 [==============================] - 2s 167us/sample - loss: 0.6939 - accuracy: 0.4610


[0.6939027951350101, 0.46095538]

If you have no idea about the training dataset,  you would expect each class with equal probability and your guess for every image would be 1/nr_of_classes. In the case of 2 classes, you predicit every image with a probability of around 0.5. The resulting cross-entropy, which is the negative log likelihood, is calculated below.

In [8]:
nr_of_classes=2
-np.log(1/nr_of_classes)

0.6931471805599453

### Return to the book 
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/Page_turn_icon_A.png" width="120" align="left" />  

Let's now work with the full MNIST dataset (all ten classes). You can use the same network architecture as before, the only thing you need to change is the number of output nodes. You use 10 output nodes, one for each class (0, 1,..., 9).

In [9]:
nb_classes = 10

# define CNN with 2 convolution blocks and 2 fully connected layers
model = Sequential()

model.add(Convolution2D(8,kernel_size,padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(8, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Convolution2D(16, kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(16,kernel_size,padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(40))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# compile model and intitialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
# summarize model along with number of model weights
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 8)         80        
_________________________________________________________________
activation_6 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 8)         584       
_________________________________________________________________
activation_7 (Activation)    (None, 28, 28, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 16)        1168      
_________________________________________________________________
activation_8 (Activation)    (None, 14, 14, 16)       

Here you predict the probabilities for all images in the training data set. You did not train the network yet, therefore the probabilities will be around 10% for each class.

In [11]:
# Calculate the probailities for the training data
Pred_prob = model.predict_proba(X_train)

In [12]:
Pred_prob[0:5]

array([[0.10749492, 0.10472608, 0.09103097, 0.09943033, 0.09795263,
        0.09219333, 0.10371377, 0.10640517, 0.09195897, 0.10509381],
       [0.10253069, 0.10584503, 0.08830696, 0.09874611, 0.09855303,
        0.09201857, 0.1070503 , 0.10808311, 0.09251422, 0.10635199],
       [0.10216265, 0.10498054, 0.09664252, 0.10318816, 0.09712867,
        0.09183189, 0.10233367, 0.10906745, 0.09367013, 0.09899428],
       [0.10977097, 0.09786078, 0.09714107, 0.09702702, 0.09982375,
        0.09594168, 0.10972438, 0.09987551, 0.09232695, 0.10050784],
       [0.10402357, 0.10470287, 0.09333142, 0.09796316, 0.09934756,
        0.0928975 , 0.1069885 , 0.10527403, 0.09193941, 0.10353192]],
      dtype=float32)

In [13]:
Pred_prob.shape, Y_train.shape

((60000, 10), (60000, 10))

### Exercise 2: Calculate the loss function using numpy
<img src="https://raw.githubusercontent.com/tensorchiefs/dl_book/master/imgs/paper-pen.png" width="60" align="left" />  

*Exercise 2: Use numpy to calculate the value of the negative log-likelihood loss (=cross entropy) that you expect for the untrained CNN, which you have constructed above to discriminate between the 10 classes. Determine the cross entropy that results from the predicted probabilities (Pred_prob). To determine the cross entropy of the prediction, you can loop over each example and use its true label (Y_train) and the predicted probability for the true class. Do you get the cross entropy value that you have expected?*




In [14]:
# Write your code here

Scroll down to see the solution.

</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>
</br>

In the next cell you calculate the cross entropy loss of each single image, then you sum up all individual losses and divide the sum with the nr of training examples. You take the negative of this result to get the NLL, also known as categorical cross entropy.

In [15]:
loss=np.zeros(len(X_train))
Y=np.argmax(Y_train,axis=1)
for i in range(0,len(X_train)):
  loss[i]=np.log(Pred_prob[i][Y[i]])
-np.sum(loss)/len(X_train)

2.3108919594844184

If you have no idea about the training dataset, your guess for every image would be 1/nr_of_classes, in the case with 10 classes, you would predicit every image with a probability around 0.1. The corresponding NLL is calculated below:

In [16]:
nr_of_classes=10
-np.log(1/nr_of_classes)

2.3025850929940455

You get more or less the same result as as you got with the model.evaluate function for the untrained CNN.  

In [17]:
model.evaluate(X_train, Y_train)

60000/60000 [==============================] - 9s 151us/sample - loss: 2.3109 - accuracy: 0.0827


[2.310891961542765, 0.08268333]